## Network creation

Using the transformed data from the previous step, create networks for February and April.

In [1]:
# to allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)

import networkx as nx
import pickle

from configuration import OUT, GRAPHS
from model.network.network_data import NetworkData
from model.network.mobility_network import MobilityNetwork
from model.network.mobility_network import MNGeneratorFromNetworkData as MNG

Define a seed for random generators.

In [2]:
SEED = 4401

Read transformed data from pickled files.

In [3]:
demographics = pickle.load(open(f'{OUT}demographics.pkl', 'rb'))

comb_counts_pre = pickle.load(open(f'{OUT}comb_counts_pre.pkl', 'rb'))
comb_counts_post = pickle.load(open(f'{OUT}comb_counts_post.pkl', 'rb'))

trip_counts_pre = pickle.load(open(f'{OUT}trip_counts_pre.pkl', 'rb'))
trip_counts_post = pickle.load(open(f'{OUT}trip_counts_post.pkl', 'rb'))

Create network data instances.

In [4]:
network_data_pre = NetworkData(demographics, comb_counts_pre, trip_counts_pre)
network_data_post = NetworkData(demographics, comb_counts_post, trip_counts_post)

network_data_post.calc_trip_count_change(network_data_pre)

In [5]:
import numpy as np
np.mean(list(network_data_post.trip_count_change.values()))

0.4316891273187813

Calculate the adjacency lists and cumulative probabilities.

In [6]:
network_data_pre.create_adjacency_list()
network_data_post.create_adjacency_list()

network_data_pre.create_cum_prob()
network_data_post.create_cum_prob()

Create the networks.

In [9]:
params_pre = dict()
params_post = dict()

params_pre[MNG.N] = params_post[MNG.N] = N = 10000
params_pre[MNG.EXPONENT] = params_post[MNG.EXPONENT] = EXPONENT = 2
params_pre[MNG.CUTOFF] = params_post[MNG.CUTOFF] = CUTOFF = 40
params_pre[MNG.SEED] = params_post[MNG.SEED] = SEED

params_pre[MNG.NETWORK_DATA] = network_data_pre
params_pre[MNG.MULTIPLIER] = False

params_post[MNG.NETWORK_DATA] = network_data_post
params_post[MNG.MULTIPLIER] = True

In [10]:
%%time

mng_pre = MNG(params=params_pre)
g_pre = mng_pre.generate()

mng_post = MNG(params=params_post)
g_post = mng_post.generate()

CPU times: user 2min 7s, sys: 207 ms, total: 2min 7s
Wall time: 2min 7s


Save the graphs to graphml format.

In [11]:
nx.write_graphml(g_pre, f'{GRAPHS}pre_N_{N}_E_{EXPONENT}_C_{CUTOFF}.graphml')
nx.write_graphml(g_post, f'{GRAPHS}post_N_{N}_E_{EXPONENT}_C_{CUTOFF}.graphml')